<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S101lemKA_unknown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing pipeline for DAAD project
## Stage 1: Lemmatization (Georgian)

# Georgian 'random' corpus 
- (collected in SketchEngine using non-topical high-frequent keywords from the top 1000 of the wikipedia list)

In [1]:
!wget https://heibox.uni-heidelberg.de/f/362c24e311104c9aa35b/?dl=1
!mv index.html?dl=1 georgianrandom02.txt

--2022-12-05 08:56:49--  https://heibox.uni-heidelberg.de/f/362c24e311104c9aa35b/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/c9ac62dc-bfe1-403d-b63a-e5b27656723a/georgianrandom02.txt [following]
--2022-12-05 08:56:51--  https://heibox.uni-heidelberg.de/seafhttp/files/c9ac62dc-bfe1-403d-b63a-e5b27656723a/georgianrandom02.txt
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 192425594 (184M) [text/plain]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 183.51M  16.4MB/s    in 12s     

2022-12-05 08:57:03 (14.9 MB/s) - ‘index.html?dl=1’ saved [192425594/192425594]



In [ ]:
%%bash
# installing TreeTagger (en, de, ka)
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper
# changing options: no-unknown, sgml, lemma
mv /content/treetagger/cmd/tree-tagger-english /content/tree-tagger-english0
awk '{ if (NR == 9) print "OPTIONS=\"-token -lemma -sgml -no-unknown\""; else print $0}' /content/tree-tagger-english0 > /content/treetagger/cmd/tree-tagger-english
chmod a+x ./treetagger/cmd/tree-tagger-english
# downloading German and Georgian 
wget https://heibox.uni-heidelberg.de/f/ec8226edebb64a359407/?dl=1
mv index.html?dl=1 /content/treetagger/lib/german-utf8.par
wget https://heibox.uni-heidelberg.de/f/9183090d2bdb41e09055/?dl=1
mv index.html?dl=1 /content/treetagger/lib/georgian.par
wget https://heibox.uni-heidelberg.de/f/9cafab0509d64ed1ac4b/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian2
cp /content/treetagger/cmd/tree-tagger-georgian2 /content/treetagger/cmd/tree-tagger-georgian
# German2 = -no-unknown 
# note: tree-tagger-german will not work, as parameter files have not been downloaded, only use tree-tagger-german2 with utf8 encoding
wget https://heibox.uni-heidelberg.de/f/acb9b8a2fa4f40e08f8a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-german2
chmod a+x /content/treetagger/cmd/tree-tagger-georgian2
chmod a+x /content/treetagger/cmd/tree-tagger-german2

In [ ]:
%%bash
wget https://heibox.uni-heidelberg.de/f/a6f7f36f175942ccad0a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian
chmod a+x /content/treetagger/cmd/tree-tagger-georgian

# cp /content/treetagger/cmd/tree-tagger-georgian2 /content/treetagger/cmd/tree-tagger-georgian


In [ ]:
# analysing German file 
# !./treetagger/cmd/tree-tagger-german2 /content/udhr/udhr_kat_v02.txt >/content/udhrTT/udhr_kat_vert.txt
!./treetagger/cmd/tree-tagger-georgian georgianrandom02.txt >georgianrandom02.vert

In [ ]:
!head -n 50 georgianrandom02.vert


In [20]:
!wc georgianrandom02.vert

 11301824  33112275 394150707 georgianrandom02.vert


In [23]:
## how to print counter in AWK:
## https://stackoverflow.com/questions/67901330/awk-how-to-print-the-last-value-of-a-counter

# !awk -F '\t' '(NF==3){printf "%s ", $3; if(FNR % 10000 == 0){printf "\n"}}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt

# if we need <p> tags for paragraphs:
# !awk -F '\t' '(NF==3){printf "%s ", $3}(NF!=3){printf "\n%s\n", $0}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt
# if we do not need paragraph tags
# !awk -F '\t' '(NF==3){printf "%s ", $3}(NF!=3){printf "\n"}' < /content/udhrTT/udhr_kat_vert.txt >/content/udhrTT/udhr_kat_lem.txt
!awk -F '\t' '($3=="<unknown>"){j++; printf "%s\n", $0} END{print j+0  > "/dev/stderr" }' <georgianrandom02.vert >georgianrandom02-unknown.txt

2123093


In [ ]:
!head -n 50 georgianrandom02-unknown.txt


In [ ]:
!tail -n 20 georgianrandom02-unknown.txt

In [18]:
D = {}
with open("georgianrandom02-unknown.txt", 'r') as f:
    for line in f:
        line = line.rstrip()
        try:
            D[line] +=1
        except:
            D[line] = 0

fo = open("georgianrandom02-unknown-frq.txt", 'w') 
for key, val in sorted(D.items(), key=lambda item: item[1], reverse=True):
    fo.write(key + '\t' + str(val) + '\n')
    

In [ ]:
!head -n 50 georgianrandom02-unknown-frq.txt


# Optional dataset (not used)
## Example: Universal declaration of human rights (

In [ ]:
# Downloading UDHR
!wget https://unicode.org/udhr/assemblies/udhr_txt.zip


In [ ]:
%%bash
mkdir udhr
mv udhr_txt.zip udhr
cd udhr/
unzip udhr_txt.zip

In [ ]:
%%bash
# put paragraph tags
awk '{print "<p>\n"$0 ; print "</p>"}' /content/udhr/udhr_kat.txt >/content/udhr/udhr_kat_v02.txt